In [181]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [182]:
df_2 = pd.read_csv('../../data/interim/data_encoded(2_of_2).csv')

In [239]:
df_1 = pd.read_csv('../../data/interim/data_encoded.csv')

In [241]:
df_1['id'] = df_1['announcement_id']

In [242]:
df = pd.merge(df_1, df_2, on='id', how='inner')

In [244]:
df = df.drop(['Unnamed: 0_x', 'announcement_id', 'building_type', 'Unnamed: 0_y'], axis=1)

In [245]:
df.loc[df['time'] > 60, 'time'] = df['time'].median()

In [246]:
df.loc[df['buildind_floor'] == 116, 'buildind_floor'] = 16

In [248]:
df = df.drop(['id', 'link'], axis=1)

In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19737 entries, 0 to 19736
Columns: 321 entries, rooms to ht_panel
dtypes: float64(286), int64(35)
memory usage: 48.3 MB


In [288]:
scaler_std = StandardScaler()

In [272]:
df_test = df.drop(['rooms_size'], axis=1)

In [290]:
X = df.drop(columns=['price'])
y = df['price']

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [299]:
X_train_scaled = scaler_std.fit_transform(X_train)
X_test_scaled = scaler_std.transform(X_test)

In [302]:
baseline_model = LinearRegression()
baseline_model.fit(X_train_scaled, y_train)
y_pred = baseline_model.predict(X_test_scaled)
baseline_mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Baseline MAPE: {baseline_mape}')

Baseline MAPE: 0.3239451801530131


In [303]:
# Sequential Feature Selector (forward selection)
sfs = SequentialFeatureSelector(baseline_model, n_features_to_select=30, direction='forward', scoring='neg_mean_absolute_percentage_error', cv=5)
sfs.fit(X_train_scaled, y_train)

# Получение отобранных признаков
selected_features = sfs.get_support()

# Создание нового набора данных с отобранными признаками
X_train_selected = X_train_scaled.loc[:, selected_features]
X_test_selected = X_test_scaled.loc[:, selected_features]

# Обучение модели на отобранных признаках
selected_model = LinearRegression()
selected_model.fit(X_train_selected, y_train)
y_pred_selected = selected_model.predict(X_test_selected)

# Оценка качества модели
selected_mape = mean_absolute_percentage_error(y_test, y_pred_selected)
print(f'MAPE на выбранных признаках: {selected_mape}')

KeyboardInterrupt: 